In [1]:
import os
import sys
import glob

In [2]:
%pwd

u'/home/dalai/carnd/data'

In [18]:
data_path = 'sim-data/sim-slack'
out_path = 'sim'

In [5]:
os.listdir(data_path)

['Red', 'None', 'Yellow', 'Green']

In [25]:
# move the files and rename
for folder in os.listdir(data_path):
    for i,filename in enumerate(os.listdir(os.path.join(data_path,folder))):
        os.rename(os.path.join(data_path,folder,filename),os.path.join(out_path,folder+str(i+1)+'.jpg'))

In [27]:
# remove the folder
%rm -R 'sim-data/sim-slack'

In [10]:
# count img files
cnt_red =  cnt_green = cnt_yellow = 0
for filename in os.listdir('sim'):
    #print(filename[0])
    if filename[0] == 'R':
        cnt_red = cnt_red + 1
    elif filename[0] == 'G':
        cnt_green = cnt_green + 1
    elif filename[0] == 'Y':
        cnt_yellow = cnt_yellow + 1
    else:
        print("WRONG")
        
print('Red:{}, Green:{}, Yellow:{}'.format(cnt_red,cnt_green,cnt_yellow))

Red:133, Green:59, Yellow:44


In [14]:
# count annotation files
cnt_red =  cnt_green = cnt_yellow = 0
for filename in os.listdir('label'):
    #print(filename[0])
    if filename[0] == 'R':
        cnt_red = cnt_red + 1
    elif filename[0] == 'G':
        cnt_green = cnt_green + 1
    elif filename[0] == 'Y':
        cnt_yellow = cnt_yellow + 1
    else:
        print('Different file:{}'.format(filename))
        
print('Red:{}, Green:{}, Yellow:{}'.format(cnt_red,cnt_green,cnt_yellow))

Different file:classes.txt
Red:133, Green:59, Yellow:44


In [6]:
from lxml import etree
import tensorflow as tf
import sys

sys.path.append("../models/research")

from object_detection.utils import dataset_util
#from object_detection.utils import label_map_util

In [7]:
xml_file = 'label/Green16.xml'

In [8]:
with tf.gfile.GFile(xml_file,'r') as fid:
    xml = fid.read()

In [9]:
xml2= etree.fromstring(xml)

In [10]:
data = dataset_util.recursive_parse_xml_to_dict(xml2)['annotation']

object_detection/utils/dataset_util.py:75: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:


In [17]:
for obj in data['object']:
    d = obj['bndbox']['xmin']
    print(d)

91
328
557


In [15]:
class TrafficLightClassifier(object):
    def __init__(self):
        PATH_TO_MODEL = 'frozen_inference_graph.pb'
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            # Works up to here.
            with tf.gfile.GFile(PATH_TO_MODEL, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
            self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
            self.d_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
            self.d_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
            self.d_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
            self.num_d = self.detection_graph.get_tensor_by_name('num_detections:0')
        self.sess = tf.Session(graph=self.detection_graph)
        
        
    def get_classification(self, img):
    # Bounding Box Detection.
    with self.detection_graph.as_default():
        # Expand dimension since the model expects image to have shape [1, None, None, 3].
        img_expanded = np.expand_dims(img, axis=0)  
        (boxes, scores, classes, num) = self.sess.run(
            [self.d_boxes, self.d_scores, self.d_classes, self.num_d],
            feed_dict={self.image_tensor: img_expanded})
    return boxes, scores, classes, num

IndentationError: expected an indented block (<ipython-input-15-9eb73b3fdbdb>, line 22)